# Black-box Optimization in Python

You've probably noticed that the SciPy optimization routines, and indeed most optimization routines, assume a continuous parameter.  However, there come cases when you would like to use a discrete parameter, such as $n$ in HW4.

There are two different strategies:
1. Pick an optimization method that accepts the parameter as an integer constraint
2. Round the parameter input within the objective function

Either of these strategies work, it is up to you.  Generally strategy 2 is easiest, since you don't need to find a specific algorithm.  Strategy 1 will require an algorithm that supports integer constraints, like `RBFOpt` (these are tricky to install sometimes).

## Black-box (derivative-free) optimization libraries

<!-- Numerical engineers really like to use Fortran, so make sure to install `flang` (Fortran language compiler) first! -->

Some of these packages are broken on Windows in `conda-forge` due to the Fortran compiler required. Numerical engineers tend to use Fortran since it's very close to the applied mathematics. So, you'll have to use `pip` to install rather than `mamba`.

Here are the different libraries that provide black-box optimization:
- `scipy.optimize` (see the differential_evolution() and dual_annealing() functions)
- `PDFO` (Powell's Derivative-Free Optimization)
- `nevergrad` (Facebook Research's Never Gradient Optimizers)
- `RBFOpt` (IBM Watson's derivative-free optimization)


## Why do we need a special optimizer for integer or black-box optimization?

Short but not technically correct answer: There is no gradient with integers (since integers are not real numbers), so gradient-based methods (most optimization methods) will not work.

The true answer is a bit more complicated.  General functions (black-box functions) are usually nonlinear, so linear optimization functions will not work.  In addition, integer constraints make any optimization problem NP-complete (as in, [nondeterministic-polynomial compute time](https://en.wikipedia.org/wiki/NP-completeness)), so the algorithms that solve these problems are usually special in some transformation of the problem.

## Example using rounding (strategy 2)

We can pretend that we only accept integers for the squaring function, and try to find the minimal x for the square function:

In [ ]:
import scipy.optimize
import matplotlib.pyplot as plt
import numpy as np
from typing import Union

In [ ]:
def square(a: Union[int, float]) -> Union[int, float]:
    return np.square(np.rint(a))

We know visually that for the base function $f(x) = x^2$ that $x = 0$ would be the minimal in both real and discrete number spaces.  So let's see if we can verify this with differential evolution:

In [ ]:
scipy.optimize.differential_evolution(square, bounds=[(-20, 20)])

What are the components of the above output?
- `x`: The optimization's final value for the parameter
- `nfev`: Number of function evaluations
- `nit`: Number of iterations
- `fun`: Value of the objective function at the `x` given

What do these outputs imply?

Let's do a slightly more complex function, such as a function that isn't so smooth.  Let's define a piecewise black-box function:

- If the input $a$ is greater than 3, the output is $a^2 + a$.
- If the input $a$ is less than -1, the output is $a^2 + a$.
- If the input $-1 \le a \le 3$, the output is $a^4 + 10$.

One can see why black-box function optimization is useful -- no assumptions are made about the function.

Let's begin by plotting the function for $-20 \le a \le 20$:

In [ ]:
def funny_piecewise(a_in: float) -> float:
    a = np.rint(a_in)
    if a > 3.0:
        return np.power(a, 2) - a
    elif a < -1.0:
        return np.power(a, 2) + a
    else:
        return np.power(a, 4) + 10

In [ ]:
x_range = np.linspace(-20, 20, 40)
y_range = [funny_piecewise(x) for x in x_range]
plt.figure(figsize=(10, 10))
plt.plot(x_range, y_range, label="funny piecewise")

We see now that the minimal is no longer 0, and is probably closer to -2.  Let's use the `differential_evolution()` function to check:

In [ ]:
scipy.optimize.differential_evolution(funny_piecewise, bounds=[(-20, 20)])

What do we end up getting?

## Example using `PDFO` on strategy 2